<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.859 · Visualización de datos · Práctica, Parte 2</p>
<p style="margin: 0; text-align:right;">2021-1 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# A9: Práctica Final (parte 2) - Wrangling data


El [**wrangling data**](https://en.wikipedia.org/wiki/Data_wrangling) es el proceso de transformar y mapear datos de un formulario de datos "sin procesar" a otro formato con la intención de hacerlo más apropiado y valioso para una variedad de propósitos posteriores, como el análisis. El objetivo del wrangling data es garantizar la calidad y la utilidad de los datos. Los analistas de datos suelen pasar la mayor parte de su tiempo en el proceso de disputa de datos en comparación con el análisis real de los datos.

El proceso de wrangling data puede incluir más manipulación, visualización de datos, agregación de datos, entrenamiento de un modelo estadístico, así como muchos otros usos potenciales. El wrangling data normalmente sigue un conjunto de pasos generales que comienzan con la extracción de los datos sin procesar de la fuente de datos, "removiendo" los datos sin procesar (por ejemplo, clasificación) o analizando los datos en estructuras de datos predefinidas y, finalmente, depositando el contenido resultante en un sumidero de datos para almacenamiento y uso futuro.
   
Para ello vamos a necesitar las siguientes librerías:

In [4]:
from six import StringIO

from IPython.display import Image 
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import pydotplus
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.max_columns', None)

# 1. Carga del conjunto de datos (1 punto)

Se ha seleccionado un conjunto de datos desde el portal Stack Overflow Annual Developer Survey, que examina todos los aspectos de la experiencia de los programadores de la comunidad (Stack Overflow), desde la satisfacción profesional y la búsqueda de empleo hasta la educación y las opiniones sobre el software de código abierto; y los resultados se publican en la siguiente URL: https://insights.stackoverflow.com/survey.

En este portal se encuentran publicados los resultados de los últimos 11 años. Para los fines de la práctica final de esta asignatura se usará el dataset del año 2021, cuyo link de descarga es: https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2021.zip.

In [3]:
so2021_df = pd.read_csv('survey_results_public.csv', header=0)
so2021_df.sample(5)

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
14490,14491,"I am not primarily a developer, but I write co...","Student, full-time",Chile,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,Other (please specify):,7,NaN,NaN,NaN,NaN,NaN,NaN,Java;Node.js;Python;R,Python,NaN,NaN,NaN,NaN,NaN,NaN,Keras;NumPy;Pandas;TensorFlow,NumPy;Pandas,Git,Git,IPython/Jupyter;Sublime Text;Visual Studio Code,IPython/Jupyter;Sublime Text,Linux-based,Visit Stack Overflow;Go for a walk or other ph...,Stack Overflow;Stack Exchange,Daily or almost daily,Yes,A few times per week,"Yes, definitely",No,25-34 years old,Man,No,Straight / Heterosexual,Hispanic or Latino/a/x,None of the above,None of the above,Appropriate in length,Easy,NaN
76987,76988,I am a developer by profession,Employed full-time,United States of America,Indiana,NaN,"Secondary school (e.g. American high school, G...",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",18,7,"Developer, full-stack","10,000 or more employees",USD\tUnited States dollar,200000.0,Yearly,Clojure;JavaScript;SQL,Clojure;JavaScript;Rust,Elasticsearch,NaN,AWS,NaN,React.js,React.js,NaN,NaN,NaN,NaN,Emacs;Sublime Text,Emacs;Sublime Text,MacOS,Go for a walk or other physical activity;Googl...,Stack Overflow;Stack Exchange,Less than once per month or monthly,No,NaN,"No, not at all",Yes,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Too long,Easy,200000.0
38902,38903,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United States of America,Texas,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,School,13,8,"Developer, full-stack","Just me - I am a freelancer, sole proprietor, ...",USD\tUnited States dollar,50000.0,Yearly,HTML/CSS;Java;JavaScript;Node.js;PHP;Python;SQ...,HTML/CSS;JavaScript;Node.js;PHP;SQL;TypeScript,Firebase;MySQL,Firebase;MySQL,AWS;Google Cloud Platform,AWS,Angular;Angular.js;Express;jQuery;Laravel,Angular;Angular.js;Express;jQuery;Laravel,Cordova,Cordova,Git,Git,Android Studio;Notepad++;Sublime Text;TextMate...,Android Studio;Visual Studio Code;Xcode,MacOS,Google it,Stack Overflow;Stack Exchange,Daily or almost daily,Yes,I have never participated in Q&A on Stack Over...,"No, not really",No,25-34 years old,Man,No,Straight / Heterosexual,South Asian,None of the above,None of the above,Appropriate in length,Easy,50000.0
78069,78070,I am a developer by profession,"Not employed, but looking for work",Bangladesh,NaN,NaN,Primary/elementary school,11 - 17 years,Coding Bootcamp;Other online resources (ex: vi...,5,NaN,"Developer, mobile;Developer, front-end;Develop...",NaN,NaN,NaN,NaN,HTML/CSS;JavaScript;Node.js;PHP;Python;SQL,C;C#;C++;HTML/CSS;Java;JavaScript;Kotlin;Node....,Firebase;MongoDB;MySQL;Oracle,Firebase;MariaDB;Microsoft SQL Server;MongoDB;...,Heroku,AWS;DigitalOcean;Google Cloud Platform;Heroku;...,Express;jQuery;Laravel;React.js;Vue.js,Angular.js;Django;Express;jQuery;Laravel;React...,NaN,NaN,Git,Git,Android Studio;Notepad++;Sublime Text;Visual S...,Android Studio;Notepad++;PyCharm;Visual Studio...,Windows,Call a coworker or friend;Visit Stack Overflow...,Stack Overflow;Stack Overflow for Teams (priva...,Multiple times per day,Yes,Multiple times per day,"Yes, definitely",No,Under 18 years old,Man,Yes,Straight / Heterosexual,Prefer 

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Selección de variables:</strong> se realiza la selección de todas las variables del dataset que servirán para responder a todas las cuestiones planteadas en la primera parte de la práctica:
</div>

In [8]:
so2021_data = so2021_df[['MainBranch', 'Employment', 'Country', 'EdLevel', 'Age1stCode', 'YearsCode', 'YearsCodePro', 'DevType', 'CompTotal', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OpSys', 'Age', 'Gender', 'Trans', 'Ethnicity', 'MentalHealth', 'ConvertedCompYearly']]
so2021_data.head(5)

,MainBranch,Employment,Country,EdLevel,Age1stCode,YearsCode,YearsCodePro,DevType,CompTotal,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith,OpSys,Age,Gender,Trans,Ethnicity,MentalHealth,ConvertedCompYearly
0,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,"Secondary school (e.g. American high school, G...",18 - 24 years,NaN,NaN,"Developer, mobile",4800.0,C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,PostgreSQL;SQLite,NaN,Laravel;Symfony,NaN,NaN,PHPStorm;Xcode,MacOS,25-34 years old,Man,No,White or of European descent,None of the above,62268.0
1,I am a student who is learning to code,"Student, full-time",Netherlands,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,7,NaN,NaN,NaN,JavaScript;Python,PostgreSQL,NaN,Angular;Flask;Vue.js,Cordova,Docker;Git;Yarn,Android Studio;IntelliJ;Notepad++;PyCharm,Windows,18-24 years old,Man,No,White or of European descent,None of the above,NaN
2,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,NaN,NaN,NaN,NaN,Assembly;C;Python;R;Rust,SQLite,Heroku,Flask,NumPy;Pandas;TensorFlow;Torch/PyTorch,NaN,IPython/Jupyter;PyCharm;RStudio;Sublime Text;V...,MacOS,18-24 years old,Man,No,Prefer not to say,None of the above,NaN
3,I am a developer by profession,Employed full-time,Austria,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,"Developer, front-end",NaN,JavaScript;TypeScript,NaN,NaN,Angular;jQuery,NaN,NaN,NaN,Windows,35-44 years old,Man,No,White or of European descent,NaN,NaN
4,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,17,10,"Developer, desktop or enterprise applications;...",NaN,Bash/Shell;HTML/CSS;Python;SQL,Elasticsearch;PostgreSQL;Redis,NaN,Flask,Apache Spark;Hadoop;NumPy;Pandas,Docker;Git;Kubernetes;Yarn,Atom;IPython/Jupyter;Notepad++;PyCharm;Vim,Linux-based,25-34 years old,Man,No,White or of European descent,NaN,NaN


In [11]:
so2021_data.shape

(83439, 23)

In [12]:
so2021_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   MainBranch                    83439 non-null  object 
 1   Employment                    83323 non-null  object 
 2   Country                       83439 non-null  object 
 3   EdLevel                       83126 non-null  object 
 4   Age1stCode                    83243 non-null  object 
 5   YearsCode                     81641 non-null  object 
 6   YearsCodePro                  61216 non-null  object 
 7   DevType                       66484 non-null  object 
 8   CompTotal                     47183 non-null  float64
 9   LanguageHaveWorkedWith        82357 non-null  object 
 10  DatabaseHaveWorkedWith        69546 non-null  object 
 11  PlatformHaveWorkedWith        52135 non-null  object 
 12  WebframeHaveWorkedWith        61707 non-null  object 
 13  M

In [13]:
so2021_data.isnull().values.any() #valores perdidos en dataset

True

In [14]:
so2021_data.isnull().any() # valores perdidos por columnas en el dataset

MainBranch                      False
Employment                       True
Country                         False
EdLevel                          True
Age1stCode                       True
YearsCode                        True
YearsCodePro                     True
DevType                          True
CompTotal                        True
LanguageHaveWorkedWith           True
DatabaseHaveWorkedWith           True
PlatformHaveWorkedWith           True
WebframeHaveWorkedWith           True
MiscTechHaveWorkedWith           True
ToolsTechHaveWorkedWith          True
NEWCollabToolsHaveWorkedWith     True
OpSys                            True
Age                              True
Gender                           True
Trans                            True
Ethnicity                        True
MentalHealth                     True
ConvertedCompYearly              True
dtype: bool

In [157]:
data = so2021_data.dropna()

In [158]:
data.isnull().any() # valores perdidos por columnas en el dataset

MainBranch                      False
Employment                      False
Country                         False
EdLevel                         False
Age1stCode                      False
YearsCode                       False
YearsCodePro                    False
DevType                         False
CompTotal                       False
LanguageHaveWorkedWith          False
DatabaseHaveWorkedWith          False
PlatformHaveWorkedWith          False
WebframeHaveWorkedWith          False
MiscTechHaveWorkedWith          False
ToolsTechHaveWorkedWith         False
NEWCollabToolsHaveWorkedWith    False
OpSys                           False
Age                             False
Gender                          False
Trans                           False
Ethnicity                       False
MentalHealth                    False
ConvertedCompYearly             False
dtype: bool

In [159]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15173 entries, 45 to 83437
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   MainBranch                    15173 non-null  object 
 1   Employment                    15173 non-null  object 
 2   Country                       15173 non-null  object 
 3   EdLevel                       15173 non-null  object 
 4   Age1stCode                    15173 non-null  object 
 5   YearsCode                     15173 non-null  object 
 6   YearsCodePro                  15173 non-null  object 
 7   DevType                       15173 non-null  object 
 8   CompTotal                     15173 non-null  float64
 9   LanguageHaveWorkedWith        15173 non-null  object 
 10  DatabaseHaveWorkedWith        15173 non-null  object 
 11  PlatformHaveWorkedWith        15173 non-null  object 
 12  WebframeHaveWorkedWith        15173 non-null  object 
 13  

In [160]:
data.head()

,MainBranch,Employment,Country,EdLevel,Age1stCode,YearsCode,YearsCodePro,DevType,CompTotal,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith,OpSys,Age,Gender,Trans,Ethnicity,MentalHealth,ConvertedCompYearly
45,I am a developer by profession,Employed full-time,Brazil,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,22,15,"Developer, desktop or enterprise applications;...",22000.0,C#;C++;JavaScript;PowerShell;SQL;TypeScript,Microsoft SQL Server;PostgreSQL;Redis,Heroku;Microsoft Azure,ASP.NET Core ;React.js,.NET Core / .NET 5,Docker;Git;Kubernetes,Notepad++;Visual Studio;Visual Studio Code,Windows,35-44 years old,Man,No,White or of European descent,I have a mood or emotional disorder (e.g. depr...,60480.0
50,I am a developer by profession,Employed full-time,Greece,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18 - 24 years,12,6,"Developer, full-stack",2000.0,C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...,Couchbase;MariaDB;Microsoft SQL Server;MongoDB...,AWS;DigitalOcean;Microsoft Azure,Angular;ASP.NET;ASP.NET Core ;Express;Svelte,.NET Framework;.NET Core / .NET 5,Docker;Kubernetes,Notepad++;Visual Studio;Visual Studio Code,Windows,25-34 years old,Man,No,White or of European descent,None of the above,25944.0
58,"I am not primarily a developer, but I write co...",Employed full-time,Russian Federation,"Professional degree (JD, MD, etc.)",11 - 17 years,5,3,"Developer, full-stack;Data scientist or machin...",120000.0,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,Oracle,Heroku,Django;FastAPI;Vue.js,NumPy;Pandas;Torch/PyTorch,Docker;Git,IPython/Jupyter;Visual Studio Code,Linux-based,25-34 years old,Man,No,White or of European descent,None of the above,22644.0
64,"I am not primarily a developer, but I write co...","Independent contractor, freelancer, or self-em...",United States of America,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,6,5,"Developer, front-end;Developer, desktop or ent...",500000.0,HTML/CSS;JavaScript;Python,MySQL,AWS,Flask,Pandas,Git,Notepad++;PyCharm;Sublime Text,Linux-based,35-44 years old,Man,No,White or of European descent,None of the above,500000.0
76,"I am not primarily a developer, but I write co...",Employed full-time,Poland,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,12,8,"Developer, front-end;Developer, full-stack;Dev...",15000.0,Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...,Firebase;Microsoft SQL Server;MongoDB;MySQL;Po...,Google Cloud Platform;Microsoft Azure,Angular;Angular.js;ASP.NET;ASP.NET Core ;Djang...,.NET Framework;.NET Core / .NET 5;Apache Spark...,Docker;Git;Unity 3D,Android Studio;Eclipse;NetBeans;Notepad++;Visu...,Linux-based,25-34 years old,Man,No,White or of European descent,None of the above,45564.0


In [174]:
data.to_csv('data.csv', index=False)

In [293]:
data_test = data.copy()

In [295]:
data_test.to_csv('data_test.csv', index=False)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable Ethnicity:</strong>.
</div>

In [296]:
from re import search

def choose_ethnia(cell_ethnia):
    val_ethnia_exceptions = ["I don't know", "Or, in your own words:"]
    
    if cell_ethnia == "I don't know;Or, in your own words:":
        return val_ethnia_exceptions[0]
    
    if search(";", cell_ethnia):
        row_ethnia_values = cell_ethnia.split(';', 5)
        first_val = row_ethnia_values[0]

        if first_val not in val_ethnia_exceptions:
            return first_val

        if len(row_ethnia_values) > 1:
            if row_ethnia_values[1] not in val_ethnia_exceptions:
                return row_ethnia_values[1]

        if len(row_ethnia_values) > 2:
            if row_ethnia_values[2] not in val_ethnia_exceptions:
                return row_ethnia_values[2]
    else:
        return cell_ethnia

In [297]:
data_test['Ethnicity'] = data_test['Ethnicity'].apply(choose_ethnia)

In [299]:
data_test.drop(index=data_test[data_test['Ethnicity'] == 'Or, in your own words:'].index, inplace=True)

In [300]:
data_test.drop(index=data_test[data_test['Ethnicity'] == 'Prefer not to say'].index, inplace=True)

In [301]:
data_test['Ethnicity'].drop_duplicates().sort_values()

7670                                             Biracial
109                           Black or of African descent
122                                            East Asian
201                                Hispanic or Latino/a/x
465                                          I don't know
4719    Indigenous (such as Native American, Pacific I...
188                                        Middle Eastern
318                                           Multiracial
243                                           South Asian
186                                       Southeast Asian
45                           White or of European descent
Name: Ethnicity, dtype: object

In [302]:
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['Black or of African descent'], 'Negro')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['East Asian'], 'Asiatico del este')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['Hispanic or Latino/a/x'], 'Latino')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(["I don't know"], 'No Definido')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['Indigenous (such as Native American, Pacific Islander, or Indigenous Australian)'], 'Indigena')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['Middle Eastern'], 'Medio Oriente')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['South Asian'], 'Asiatico del Sur')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['Southeast Asian'], 'Asiatico del Sudeste')
data_test['Ethnicity'] = data_test['Ethnicity'].replace(['White or of European descent'], 'Blanco o Europeo')

In [303]:
data_test['Ethnicity'].drop_duplicates().sort_values()

186     Asiatico del Sudeste
243         Asiatico del Sur
122        Asiatico del este
7670                Biracial
45          Blanco o Europeo
4719                Indigena
201                   Latino
188            Medio Oriente
318              Multiracial
109                    Negro
465              No Definido
Name: Ethnicity, dtype: object

In [304]:
data_test.to_csv('data_test.csv', index=False)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable Employment:</strong>.
</div>

In [305]:
data_test['Employment'].drop_duplicates().sort_values()

45                                   Employed full-time
83                                   Employed part-time
64    Independent contractor, freelancer, or self-em...
Name: Employment, dtype: object

In [306]:
data_test['Employment'] = data_test['Employment'].replace(['Employed full-time'], 'Tiempo completo')
data_test['Employment'] = data_test['Employment'].replace(['Employed part-time'], 'Tiempo parcial')
data_test['Employment'] = data_test['Employment'].replace(['Independent contractor, freelancer, or self-employed'], 'Independiete')

In [307]:
data_test['Employment'].drop_duplicates().sort_values()

64       Independiete
45    Tiempo completo
83     Tiempo parcial
Name: Employment, dtype: object

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable EdLevel:</strong>.
</div>

In [308]:
data_test['EdLevel'].drop_duplicates().sort_values()

130                  Associate degree (A.A., A.S., etc.)
45          Bachelor’s degree (B.A., B.S., B.Eng., etc.)
64       Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
77            Other doctoral degree (Ph.D., Ed.D., etc.)
731                            Primary/elementary school
58                    Professional degree (JD, MD, etc.)
380    Secondary school (e.g. American high school, G...
110    Some college/university study without earning ...
86                                        Something else
Name: EdLevel, dtype: object

In [309]:
data_test['EdLevel'] = data_test['EdLevel'].replace(['Associate degree (A.A., A.S., etc.)'], 'Grado Asociado')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Bachelor’s degree (B.A., B.S., B.Eng., etc.)'], 'Licenciatura')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)'], 'Master')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Other doctoral degree (Ph.D., Ed.D., etc.)'], 'Doctorado')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Primary/elementary school'], 'Primaria')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Professional degree (JD, MD, etc.)'], 'Grado Profesional')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)'], 'Secundaria')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Some college/university study without earning a degree'], 'Estudios sin grado')
data_test['EdLevel'] = data_test['EdLevel'].replace(['Something else'], 'Otro')

In [310]:
data_test['EdLevel'].drop_duplicates().sort_values()

77              Doctorado
110    Estudios sin grado
130        Grado Asociado
58      Grado Profesional
45           Licenciatura
64                 Master
86                   Otro
731              Primaria
380            Secundaria
Name: EdLevel, dtype: object

In [311]:
data_test.to_csv('data_test.csv', index=False)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable DevType:</strong>.
</div>

In [312]:
data_test['DevType'].drop_duplicates().sort_values()

6113                                   Academic researcher
1413          Academic researcher;Data or business analyst
9267     Academic researcher;Database administrator;Dev...
37989    Academic researcher;Database administrator;Sci...
8378                 Academic researcher;DevOps specialist
                               ...                        
27480                                     Student;Educator
23376                         Student;System administrator
77415                Student;System administrator;Educator
1317                                  System administrator
14465                 System administrator;Product manager
Name: DevType, Length: 3374, dtype: object

In [313]:
from re import search

def choose_devtype(cell_devtype):
    val_devtype_exceptions = ["Other (please specify):"]
    
    if cell_devtype == "Other (please specify):":
        return val_devtype_exceptions[0]
    
    if search(";", cell_devtype):
        row_devtype_values = cell_devtype.split(';', 10)
        first_val = row_devtype_values[0]
        
        if first_val not in val_devtype_exceptions:
            return first_val

        if len(row_devtype_values) > 1:
            if row_devtype_values[1] not in val_devtype_exceptions:
                return row_devtype_values[1]    

    else:
        return cell_devtype

In [314]:
data_test['DevType'] = data_test['DevType'].apply(choose_devtype)

In [315]:
data_test['DevType'].head()

45    Developer, desktop or enterprise applications
50                            Developer, full-stack
58                            Developer, full-stack
64                             Developer, front-end
76                             Developer, front-end
Name: DevType, dtype: object

In [316]:
data_test['DevType'].drop_duplicates().sort_values()

1160                               Academic researcher
4752                          Data or business analyst
77       Data scientist or machine learning specialist
6237                            Database administrator
21365                                         Designer
4288                                 DevOps specialist
1348                             Developer, QA or test
86                                 Developer, back-end
45       Developer, desktop or enterprise applications
2517       Developer, embedded applications or devices
64                                Developer, front-end
50                               Developer, full-stack
690                        Developer, game or graphics
100                                  Developer, mobile
15061                                         Educator
114                                     Engineer, data
3419                        Engineer, site reliability
942                                Engineering manager
17240     

In [317]:
data_test['DevType'].value_counts()

Developer, full-stack                            4416
Developer, front-end                             2903
Developer, mobile                                2798
Developer, back-end                              1484
Developer, desktop or enterprise applications    1096
Engineer, data                                    595
Data scientist or machine learning specialist     408
Other (please specify):                           137
Engineering manager                               126
DevOps specialist                                 107
Senior Executive (C-Suite, VP, etc.)               74
Academic researcher                                62
Developer, QA or test                              59
Data or business analyst                           48
Developer, embedded applications or devices        41
System administrator                               25
Engineer, site reliability                         24
Product manager                                    23
Database administrator      

In [318]:
data_test['DevType'] = data_test['DevType'].replace(['Developer, full-stack'], 'Desarrollador full-stack')
data_test['DevType'] = data_test['DevType'].replace(['Developer, front-end'], 'Desarrollador front-end')
data_test['DevType'] = data_test['DevType'].replace(['Developer, mobile'], 'Desarrollador móvil')
data_test['DevType'] = data_test['DevType'].replace(['Developer, back-end'], 'Desarrollador back-end')
data_test['DevType'] = data_test['DevType'].replace(['Developer, desktop or enterprise applications'], 'Desarrollador Escritorio')
data_test['DevType'] = data_test['DevType'].replace(['Engineer, data'], 'Ingeniero de datos')
data_test['DevType'] = data_test['DevType'].replace(['Data scientist or machine learning specialist'], 'Cientifico de datos')
data_test['DevType'] = data_test['DevType'].replace(['Other (please specify):'], 'Otro')
data_test['DevType'] = data_test['DevType'].replace(['Engineering manager'], 'Manager de Ingeniería')
data_test['DevType'] = data_test['DevType'].replace(['DevOps specialist'], 'Especialista en DevOps')
data_test['DevType'] = data_test['DevType'].replace(['Senior Executive (C-Suite, VP, etc.)'], 'Ejecutivo Senior')
data_test['DevType'] = data_test['DevType'].replace(['Academic researcher'], 'Investigador Académico')
data_test['DevType'] = data_test['DevType'].replace(['Developer, QA or test'], 'Desarrollador de QA o Test')
data_test['DevType'] = data_test['DevType'].replace(['Data or business analyst'], 'Analista de datos o negocio')
data_test['DevType'] = data_test['DevType'].replace(['Developer, embedded applications or devices'], 'Desarrollador de aplicaciones embebidas')
data_test['DevType'] = data_test['DevType'].replace(['System administrator'], 'Administrador de sistemas')
data_test['DevType'] = data_test['DevType'].replace(['Engineer, site reliability'], 'Ingeniero de confiabilidad del sitio')
data_test['DevType'] = data_test['DevType'].replace(['Product manager'], 'Gerente de producto')
data_test['DevType'] = data_test['DevType'].replace(['Database administrator'], 'Administrador de base de datos')
data_test['DevType'] = data_test['DevType'].replace(['Student'], 'Estudiante')
data_test['DevType'] = data_test['DevType'].replace(['Developer, game or graphics'], 'Desarrollador de juegos o gráfico')
data_test['DevType'] = data_test['DevType'].replace(['Scientist'], 'Científico')
data_test['DevType'] = data_test['DevType'].replace(['Designer'], 'Diseñador')
data_test['DevType'] = data_test['DevType'].replace(['Educator'], 'Educador')
data_test['DevType'] = data_test['DevType'].replace(['Marketing or sales professional'], 'Profesional en Marketing o ventas')

In [319]:
data_test['DevType'].drop_duplicates().sort_values()

6237              Administrador de base de datos
1317                   Administrador de sistemas
4752                 Analista de datos o negocio
77                           Cientifico de datos
5724                                  Científico
45                      Desarrollador Escritorio
86                        Desarrollador back-end
1348                  Desarrollador de QA o Test
2517     Desarrollador de aplicaciones embebidas
690            Desarrollador de juegos o gráfico
64                       Desarrollador front-end
50                      Desarrollador full-stack
100                          Desarrollador móvil
21365                                  Diseñador
15061                                   Educador
710                             Ejecutivo Senior
4288                      Especialista en DevOps
9664                                  Estudiante
28419                        Gerente de producto
3419        Ingeniero de confiabilidad del sitio
114                 

In [320]:
data_test['DevType'].value_counts()

Desarrollador full-stack                   4416
Desarrollador front-end                    2903
Desarrollador móvil                        2798
Desarrollador back-end                     1484
Desarrollador Escritorio                   1096
Ingeniero de datos                          595
Cientifico de datos                         408
Otro                                        137
Manager de Ingeniería                       126
Especialista en DevOps                      107
Ejecutivo Senior                             74
Investigador Académico                       62
Desarrollador de QA o Test                   59
Analista de datos o negocio                  48
Desarrollador de aplicaciones embebidas      41
Administrador de sistemas                    25
Ingeniero de confiabilidad del sitio         24
Gerente de producto                          23
Administrador de base de datos               20
Estudiante                                   16
Desarrollador de juegos o gráfico       

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable MainBranch:</strong>
</div>

In [321]:
data_test['MainBranch'].drop_duplicates().sort_values()

45                       I am a developer by profession
58    I am not primarily a developer, but I write co...
Name: MainBranch, dtype: object

In [322]:
data_test['MainBranch'] = data_test['MainBranch'].replace(['I am a developer by profession'], 'Desarrollador Profesional')
data_test['MainBranch'] = data_test['MainBranch'].replace(['I am not primarily a developer, but I write code sometimes as part of my work'], 'Desarrollador ocasional')

In [323]:
data_test['MainBranch'].drop_duplicates().sort_values()

45    Desarrollador Profesional
58      Desarrollador ocasional
Name: MainBranch, dtype: object

In [324]:
data_test.to_csv('data_test.csv', index=False)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable Age1stCode:</strong>
</div>

In [325]:
data_test['Age1stCode'].drop_duplicates().sort_values()

45              11 - 17 years
50              18 - 24 years
222             25 - 34 years
751             35 - 44 years
2371            45 - 54 years
77               5 - 10 years
2225            55 - 64 years
37610     Older than 64 years
188      Younger than 5 years
Name: Age1stCode, dtype: object

In [326]:
data_test['Age1stCode'].value_counts()

11 - 17 years           8018
18 - 24 years           3408
5 - 10 years            2018
25 - 34 years            639
35 - 44 years            219
Younger than 5 years     105
45 - 54 years             85
55 - 64 years             24
Older than 64 years        1
Name: Age1stCode, dtype: int64

In [327]:
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['11 - 17 years'], '11-17')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['18 - 24 years'], '18-24')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['25 - 34 years'], '25-34')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['35 - 44 years'], '35-44')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['45 - 54 years'], '45-54')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['5 - 10 years'], '5-10')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['55 - 64 years'], '55-64')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['Older than 64 years'], '> 64')
data_test['Age1stCode'] = data_test['Age1stCode'].replace(['Younger than 5 years'], '< 5')

In [328]:
data_test['Age1stCode'].value_counts()

11-17    8018
18-24    3408
5-10     2018
25-34     639
35-44     219
< 5       105
45-54      85
55-64      24
> 64        1
Name: Age1stCode, dtype: int64

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable YearsCode:</strong>
</div>

In [333]:
data_test['YearsCode'] = data_test['YearsCode'].replace(['More than 50 years'], 50)
data_test['YearsCode'] = data_test['YearsCode'].replace(['Less than 1 year'], 1)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable YearsCodePro:</strong>
</div>

In [334]:
data_test['YearsCodePro'] = data_test['YearsCodePro'].replace(['More than 50 years'], 50)
data_test['YearsCodePro'] = data_test['YearsCodePro'].replace(['Less than 1 year'], 1)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable OpSys:</strong>
</div>

In [335]:
data_test['OpSys'].value_counts()

Windows                              6770
MacOS                                4255
Linux-based                          2912
Windows Subsystem for Linux (WSL)     523
Other (please specify):                47
BSD                                    10
Name: OpSys, dtype: int64

In [336]:
data_test['OpSys'] = data_test['OpSys'].replace(['Windows Subsystem for Linux (WSL)'], 'Windows')
data_test['OpSys'] = data_test['OpSys'].replace(['Linux-based'], 'Linux')
data_test['OpSys'] = data_test['OpSys'].replace(['Other (please specify)'], 'Otro')

In [337]:
data_test['OpSys'].value_counts()

Windows                    7293
MacOS                      4255
Linux                      2912
Other (please specify):      47
BSD                          10
Name: OpSys, dtype: int64

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable Age:</strong>
</div>

In [338]:
data_test['Age'].value_counts()

25-34 years old       7275
35-44 years old       3361
18-24 years old       2602
45-54 years old        957
55-64 years old        255
Under 18 years old      35
65 years or older       26
Prefer not to say        6
Name: Age, dtype: int64

In [339]:
data_test['Age'] = data_test['Age'].replace(['25-34 years old'], '25-34')
data_test['Age'] = data_test['Age'].replace(['35-44 years old'], '35-44')
data_test['Age'] = data_test['Age'].replace(['18-24 years old'], '18-24')
data_test['Age'] = data_test['Age'].replace(['45-54 years old'], '45-54')
data_test['Age'] = data_test['Age'].replace(['55-64 years old'], '55-64')
data_test['Age'] = data_test['Age'].replace(['Under 18 years old'], '< 18')
data_test['Age'] = data_test['Age'].replace(['65 years or older'], '>= 65')
data_test['Age'] = data_test['Age'].replace(['Prefer not to say'], 'No definido')

In [362]:
data_test['Age'] = data_test['Age'].replace(['25-34 years old'], '25-34')

In [363]:
data_test['Age'].value_counts()

25-34          7275
35-44          3361
18-24          2602
45-54           957
55-64           255
< 18             35
>= 65            26
No definido       6
Name: Age, dtype: int64

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable Gender:</strong>
</div>

In [341]:
data_test['Gender'].value_counts()

Man                                                                                   13748
Woman                                                                                   502
Non-binary, genderqueer, or gender non-conforming                                        94
Prefer not to say                                                                        53
Man;Non-binary, genderqueer, or gender non-conforming                                    37
Man;Or, in your own words:                                                               27
Or, in your own words:                                                                   25
Woman;Non-binary, genderqueer, or gender non-conforming                                  19
Man;Woman                                                                                 5
Man;Woman;Non-binary, genderqueer, or gender non-conforming;Or, in your own words:        3
Non-binary, genderqueer, or gender non-conforming;Or, in your own words:        

In [342]:
data_test['Gender'] = data_test['Gender'].replace(['Man'], 'Hombre')
data_test['Gender'] = data_test['Gender'].replace(['Woman'], 'Mujer')
data_test['Gender'] = data_test['Gender'].replace(['Non-binary, genderqueer, or gender non-conforming'], 'No binario u otro')
data_test['Gender'] = data_test['Gender'].replace(['Man;Non-binary, genderqueer, or gender non-conforming'], 'No binario u otro')
data_test['Gender'] = data_test['Gender'].replace(['Man;Or, in your own words:'], 'Hombre')
data_test['Gender'] = data_test['Gender'].replace(['Or, in your own words:'], 'No definido')
data_test['Gender'] = data_test['Gender'].replace(['Woman;Non-binary, genderqueer, or gender non-conforming'], 'No binario u otro')
data_test['Gender'] = data_test['Gender'].replace(['Man;Woman'], 'No definido')
data_test['Gender'] = data_test['Gender'].replace(['Man;Woman;Non-binary, genderqueer, or gender non-conforming;Or, in your own words:'], 'No binario u otro')
data_test['Gender'] = data_test['Gender'].replace(['Non-binary, genderqueer, or gender non-conforming;Or, in your own words:'], 'No binario u otro')
data_test['Gender'] = data_test['Gender'].replace(['Man;Woman;Non-binary, genderqueer, or gender non-conforming'], 'No binario u otro')

In [344]:
data_test['Gender'] = data_test['Gender'].replace(['Prefer not to say'], 'No definido')

In [350]:
data_test['Gender'].value_counts()

Hombre               13775
Mujer                  502
No binario u otro      157
No definido             83
Name: Gender, dtype: int64

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable Trans:</strong>
</div>

In [349]:
data_test['Trans'].value_counts()

No                        14262
Yes                         110
Prefer not to say            88
Or, in your own words:       57
Name: Trans, dtype: int64

In [351]:
data_test['Trans'] = data_test['Trans'].replace(['Yes'], 'Si')
data_test['Trans'] = data_test['Trans'].replace(['Prefer not to say'], 'No definido')
data_test['Trans'] = data_test['Trans'].replace(['Or, in your own words:'], 'No definido')

In [352]:
data_test['Trans'].value_counts()

No             14262
No definido      145
Si               110
Name: Trans, dtype: int64

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Variable MentalHealth:</strong>
</div>

In [353]:
data_test['MentalHealth'].value_counts()

None of the above                                                                                                                                                                                                                                    10924
I have a concentration and/or memory disorder (e.g. ADHD)                                                                                                                                                                                              627
I have an anxiety disorder                                                                                                                                                                                                                             605
I have a mood or emotional disorder (e.g. depression, bipolar disorder)                                                                                                                                                                                

In [356]:
from re import search

def choose_mental_health(cell_mental_health):
    val_mental_health_exceptions = ["Or, in your own words:"]
    
    if cell_mental_health == "Or, in your own words:":
        return val_mental_health_exceptions[0]
    
    if search(";", cell_mental_health):
        row_mental_health_values = cell_mental_health.split(';', 10)
        first_val = row_mental_health_values[0]
        
        return first_val
    else:
        return cell_mental_health

In [357]:
data_test['MentalHealth'] = data_test['MentalHealth'].apply(choose_mental_health)

In [358]:
data_test['MentalHealth'].value_counts()

None of the above                                                          10924
I have a concentration and/or memory disorder (e.g. ADHD)                   1303
I have a mood or emotional disorder (e.g. depression, bipolar disorder)      908
I have an anxiety disorder                                                   637
Prefer not to say                                                            396
I have autism / an autism spectrum disorder (e.g. Asperger's)                207
Or, in your own words:                                                       142
Name: MentalHealth, dtype: int64

In [359]:
data_test['MentalHealth'] = data_test['MentalHealth'].replace(['None of the above'], 'Ninguna de las mencionadas')
data_test['MentalHealth'] = data_test['MentalHealth'].replace(['I have a concentration and/or memory disorder (e.g. ADHD)'], 'Desorden de concentración o memoria')
data_test['MentalHealth'] = data_test['MentalHealth'].replace(['I have a mood or emotional disorder (e.g. depression, bipolar disorder)'], 'Desorden emocional')
data_test['MentalHealth'] = data_test['MentalHealth'].replace(['I have an anxiety disorder'], 'Desorden de ansiedad')
data_test['MentalHealth'] = data_test['MentalHealth'].replace(['Prefer not to say'], 'No definido')
data_test['MentalHealth'] = data_test['MentalHealth'].replace(["I have autism / an autism spectrum disorder (e.g. Asperger's)"], 'Tipo de autismo')
data_test['MentalHealth'] = data_test['MentalHealth'].replace(['Or, in your own words:'], 'No definido')

In [360]:
data_test['MentalHealth'].value_counts()

Ninguna de las mencionadas             10924
Desorden de concentración o memoria     1303
Desorden emocional                       908
Desorden de ansiedad                     637
No definido                              538
Tipo de autismo                          207
Name: MentalHealth, dtype: int64

# 2. Selección de campos para subdatasets

Se seleccionarán los campos adecuados para responder a cada una de las cuestiones que se plantearon en la primera parte de la práctica.

### 2.1. Según la autodeterminación de la etnia, ¿Qué etnia tiene un mayor sueldo anual?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [366]:
data_etnia = data_test[['Country', 'Ethnicity', 'ConvertedCompYearly']]
data_etnia.head()

,Country,Ethnicity,ConvertedCompYearly
45,Brazil,Blanco o Europeo,60480.0
50,Greece,Blanco o Europeo,25944.0
58,Russian Federation,Blanco o Europeo,22644.0
64,United States of America,Blanco o Europeo,500000.0
76,Poland,Blanco o Europeo,45564.0


In [509]:
df_data_etnia = data_etnia.copy()

In [512]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_etnia['ConvertedCompYearly'], 0.1)

print(df_data_etnia[mask])

                        Country             Ethnicity  ConvertedCompYearly
45                       Brazil      Blanco o Europeo              60480.0
50                       Greece      Blanco o Europeo              25944.0
58           Russian Federation      Blanco o Europeo              22644.0
76                       Poland      Blanco o Europeo              45564.0
77                       Canada      Blanco o Europeo             151263.0
...                         ...                   ...                  ...
83425                   Finland      Blanco o Europeo              19452.0
83428                    Brazil                Latino              41232.0
83431                  Pakistan  Asiatico del Sudeste              11676.0
83432                    Canada     Asiatico del este              80169.0
83436  United States of America      Blanco o Europeo              90000.0

[11611 rows x 3 columns]


In [513]:
df_data_etnia_no_outliers = df_data_etnia[mask]

In [517]:
df_data_etnia_no_outliers = df_data_etnia_no_outliers.copy()

In [519]:
df_data_etnia_no_outliers['ConvertedCompYearlyCategorical'] = 'ALTO'
df_data_etnia_no_outliers.loc[(df_data_etnia_no_outliers['ConvertedCompYearly'] >= 0) & (df_data_etnia_no_outliers['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_data_etnia_no_outliers.loc[(df_data_etnia_no_outliers['ConvertedCompYearly'] > 32747) & (df_data_etnia_no_outliers['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_data_etnia_no_outliers)

                        Country             Ethnicity  ConvertedCompYearly  \
45                       Brazil      Blanco o Europeo              60480.0   
50                       Greece      Blanco o Europeo              25944.0   
58           Russian Federation      Blanco o Europeo              22644.0   
76                       Poland      Blanco o Europeo              45564.0   
77                       Canada      Blanco o Europeo             151263.0   
...                         ...                   ...                  ...   
83425                   Finland      Blanco o Europeo              19452.0   
83428                    Brazil                Latino              41232.0   
83431                  Pakistan  Asiatico del Sudeste              11676.0   
83432                    Canada     Asiatico del este              80169.0   
83436  United States of America      Blanco o Europeo              90000.0   

      ConvertedCompYearlyCategorical  
45                      

In [520]:
df_data_etnia_alto = df_data_etnia_no_outliers[df_data_etnia_no_outliers['ConvertedCompYearlyCategorical'] == 'ALTO']

In [521]:
df_data_etnia_alto = df_data_etnia_alto[['Ethnicity', 'ConvertedCompYearlyCategorical']]

In [523]:
df_flourish = df_data_etnia_alto['Ethnicity'].value_counts().to_frame('counts').reset_index()

In [524]:
df_flourish

,index,counts
0,Blanco o Europeo,2413
1,Latino,119
2,Asiatico del Sur,97
3,Medio Oriente,75
4,Asiatico del este,51
5,Negro,44
6,Asiatico del Sudeste,43
7,Multiracial,23
8,No Definido,15
9,Biracial,9


In [525]:
df_flourish.to_csv('001_df_flourish.csv', index=False)

In [443]:
df_data_etnia_alto.to_csv('001_df_data_etnia_alto.csv', index=False)

In [439]:
df_data_etnia.to_csv('001_data_etnia_categorical.csv', index=False)

In [367]:
data_etnia.to_csv('001_data_etnia.csv', index=False)

### 2.2. ¿Cuáles son los porcentajes de programadores que trabajan a tiempo completo, medio tiempo o freelance?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [368]:
data_time_work_dev = data_test[['Country', 'Employment', 'ConvertedCompYearly', 'EdLevel', 'Age']]
data_time_work_dev.head()

,Country,Employment,ConvertedCompYearly,EdLevel,Age
45,Brazil,Tiempo completo,60480.0,Licenciatura,35-44
50,Greece,Tiempo completo,25944.0,Licenciatura,25-34
58,Russian Federation,Tiempo completo,22644.0,Grado Profesional,25-34
64,United States of America,Independiete,500000.0,Master,35-44
76,Poland,Tiempo completo,45564.0,Licenciatura,25-34


In [448]:
df_flourish_002 = data_time_work_dev['Employment'].value_counts().to_frame('counts').reset_index()

In [449]:
df_flourish_002

,index,counts
0,Tiempo completo,12402
1,Independiete,1678
2,Tiempo parcial,437


In [454]:
df_flourish_002['counts'] = (df_flourish_002['counts'] * 100 ) / data_time_work_dev.shape[0]

In [455]:
df_flourish_002

,index,counts
0,Tiempo completo,85.430874
1,Independiete,11.558862
2,Tiempo parcial,3.010264


In [456]:
df_flourish_002['counts'] = df_flourish_002['counts'].round(2)

In [457]:
df_flourish_002

,index,counts
0,Tiempo completo,85.43
1,Independiete,11.56
2,Tiempo parcial,3.01


In [458]:
df_flourish_002.to_csv('002_df_flourish.csv', index=False)

### 2.3. ¿Cuáles son los países con mayor número de programadores profesionales que son activos en la comunidad Stack Overflow?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [370]:
data_pro_dev_active_so = data_test[['Country', 'Employment', 'MainBranch', 'EdLevel', 'DevType', 'Age']]
data_pro_dev_active_so.head()

,Country,Employment,MainBranch,EdLevel,DevType,Age
45,Brazil,Tiempo completo,Desarrollador Profesional,Licenciatura,Desarrollador Escritorio,35-44
50,Greece,Tiempo completo,Desarrollador Profesional,Licenciatura,Desarrollador full-stack,25-34
58,Russian Federation,Tiempo completo,Desarrollador ocasional,Grado Profesional,Desarrollador full-stack,25-34
64,United States of America,Independiete,Desarrollador ocasional,Master,Desarrollador front-end,35-44
76,Poland,Tiempo completo,Desarrollador ocasional,Licenciatura,Desarrollador front-end,25-34


In [464]:
df_flourish_003 = data_pro_dev_active_so['Country'].value_counts().sort_values(ascending=False).head(10)

In [477]:
df_flourish_003 = df_flourish_003.to_frame()

In [482]:
df_flourish_003 = df_flourish_003.reset_index()
df_flourish_003.columns = ["País", "# Programadores Profesionales"]

In [485]:
df_flourish_003.to_csv('003_df_flourish_003.csv', index=False)

### 2.4. ¿Cuál es el nivel educativo que mayores ingresos registra entre los encuestados?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [495]:
data_edlevel_income = data_test[['ConvertedCompYearly', 'EdLevel']]
data_edlevel_income.head()

,ConvertedCompYearly,EdLevel
45,60480.0,Licenciatura
50,25944.0,Licenciatura
58,22644.0,Grado Profesional
64,500000.0,Master
76,45564.0,Licenciatura


In [501]:
df_data_edlevel_income = data_edlevel_income.copy()

In [502]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_edlevel_income['ConvertedCompYearly'], 0.1)

print(df_data_edlevel_income[mask])

       ConvertedCompYearly            EdLevel
45                 60480.0       Licenciatura
50                 25944.0       Licenciatura
58                 22644.0  Grado Profesional
76                 45564.0       Licenciatura
77                151263.0          Doctorado
...                    ...                ...
83425              19452.0         Secundaria
83428              41232.0             Master
83431              11676.0       Licenciatura
83432              80169.0       Licenciatura
83436              90000.0         Secundaria

[11611 rows x 2 columns]


In [503]:
df_data_edlevel_income = df_data_edlevel_income[mask]

In [505]:
df_data_edlevel_income['ConvertedCompYearlyCategorical'] = 'ALTO'
df_data_edlevel_income.loc[(df_data_edlevel_income['ConvertedCompYearly'] >= 0) & (df_data_edlevel_income['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_data_edlevel_income.loc[(df_data_edlevel_income['ConvertedCompYearly'] > 32747) & (df_data_edlevel_income['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_data_edlevel_income)

       ConvertedCompYearly            EdLevel ConvertedCompYearlyCategorical
45                 60480.0       Licenciatura                          MEDIO
50                 25944.0       Licenciatura                           BAJO
58                 22644.0  Grado Profesional                           BAJO
76                 45564.0       Licenciatura                          MEDIO
77                151263.0          Doctorado                           ALTO
...                    ...                ...                            ...
83425              19452.0         Secundaria                           BAJO
83428              41232.0             Master                          MEDIO
83431              11676.0       Licenciatura                           BAJO
83432              80169.0       Licenciatura                          MEDIO
83436              90000.0         Secundaria                          MEDIO

[11611 rows x 3 columns]


In [506]:
df_data_edlevel_income = df_data_edlevel_income[df_data_edlevel_income['ConvertedCompYearlyCategorical'] == 'ALTO']

In [507]:
df_data_edlevel_income = df_data_edlevel_income[['EdLevel', 'ConvertedCompYearlyCategorical']]

In [527]:
df_flourish_004 = df_data_edlevel_income['EdLevel'].value_counts().to_frame('counts').reset_index()

In [528]:
df_flourish_004

,index,counts
0,Licenciatura,1481
1,Master,715
2,Estudios sin grado,356
3,Grado Asociado,117
4,Doctorado,96
5,Secundaria,80
6,Grado Profesional,21
7,Primaria,13
8,Otro,13


In [529]:
df_flourish_004.to_csv('004_df_flourish.csv', index=False)

### 2.5. ¿Existe brecha salarial entre hombres y mujeres u otros géneros?, y de ¿Cuánto es la diferencia? ¿Cuáles son los peores países en cuanto a brecha salarial? ¿Cuáles son los países que han reducido esta brecha salarial entre programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [585]:
data_wage_gap = data_test[['Country', 'ConvertedCompYearly', 'Gender']]
data_wage_gap.head()

,Country,ConvertedCompYearly,Gender
45,Brazil,60480.0,Hombre
50,Greece,25944.0,Hombre
58,Russian Federation,22644.0,Hombre
64,United States of America,500000.0,Hombre
76,Poland,45564.0,Hombre


In [587]:
df_data_wage_gap = data_wage_gap.copy()

In [588]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_wage_gap['ConvertedCompYearly'], 0.1)

print(df_data_wage_gap[mask])

                        Country  ConvertedCompYearly  Gender
45                       Brazil              60480.0  Hombre
50                       Greece              25944.0  Hombre
58           Russian Federation              22644.0  Hombre
76                       Poland              45564.0  Hombre
77                       Canada             151263.0  Hombre
...                         ...                  ...     ...
83425                   Finland              19452.0  Hombre
83428                    Brazil              41232.0  Hombre
83431                  Pakistan              11676.0  Hombre
83432                    Canada              80169.0   Mujer
83436  United States of America              90000.0  Hombre

[11611 rows x 3 columns]


In [589]:
df_data_wage_gap = df_data_wage_gap[mask]

In [591]:
df_data_wage_gap['ConvertedCompYearlyCategorical'] = 'ALTO'
df_data_wage_gap.loc[(df_data_wage_gap['ConvertedCompYearly'] >= 0) & (df_data_wage_gap['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_data_wage_gap.loc[(df_data_wage_gap['ConvertedCompYearly'] > 32747) & (df_data_wage_gap['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_data_wage_gap)

                        Country  ConvertedCompYearly  Gender  \
45                       Brazil              60480.0  Hombre   
50                       Greece              25944.0  Hombre   
58           Russian Federation              22644.0  Hombre   
76                       Poland              45564.0  Hombre   
77                       Canada             151263.0  Hombre   
...                         ...                  ...     ...   
83425                   Finland              19452.0  Hombre   
83428                    Brazil              41232.0  Hombre   
83431                  Pakistan              11676.0  Hombre   
83432                    Canada              80169.0   Mujer   
83436  United States of America              90000.0  Hombre   

      ConvertedCompYearlyCategorical  
45                             MEDIO  
50                              BAJO  
58                              BAJO  
76                             MEDIO  
77                              ALTO

In [592]:
df_data_wage_gap = df_data_wage_gap[df_data_wage_gap['ConvertedCompYearlyCategorical'].isin(['ALTO', 'MEDIO'])]

In [593]:
df_data_wage_gap = df_data_wage_gap[['Country', 'Gender', 'ConvertedCompYearlyCategorical']]

In [595]:
df_data_wage_gap.to_csv('005_df_data_wage_gap.csv', index=False)

In [572]:
df_data_wage_gap['ConvertedCompYearlyCategorical'].drop_duplicates().sort_values()

77     ALTO
45    MEDIO
Name: ConvertedCompYearlyCategorical, dtype: object

In [573]:
df_data_wage_gap['Gender'].drop_duplicates().sort_values()

45                 Hombre
264                 Mujer
702     No binario u otro
2559          No definido
Name: Gender, dtype: object

In [574]:
df_data_wage_gap['Country'].drop_duplicates().sort_values()

27198                             Afghanistan
54847                                 Albania
25364                                 Algeria
34843                                 Andorra
289                                 Argentina
                         ...                 
128                  United States of America
1759                                  Uruguay
44422    Venezuela, Bolivarian Republic of...
10617                                Viet Nam
27638                                  Zambia
Name: Country, Length: 126, dtype: object

In [575]:
df_data_wage_gap1 = df_data_wage_gap.copy()

In [615]:
df_flourish_005 = df_data_wage_gap1.groupby(['Country', 'Gender']).size().unstack(fill_value=0).sort_values('Hombre')

In [616]:
df_flourish_005 = df_flourish_005.apply(lambda x: pd.concat([x.head(40), x.tail(5)]))

In [609]:
df_flourish_005.to_csv('005_flourish_data.csv', index=True)

### 2.6. ¿Cuáles son los ingresos promedios según los rangos de edad? ¿Cuál es el rango de edad con el mejor y peor ingreso?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [618]:
data_age_income = data_test[['ConvertedCompYearly', 'Age']]
data_age_income.head()

,ConvertedCompYearly,Age
45,60480.0,35-44
50,25944.0,25-34
58,22644.0,25-34
64,500000.0,35-44
76,45564.0,25-34


In [619]:
df_data_age_income = data_age_income.copy()

In [620]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_age_income['ConvertedCompYearly'], 0.1)

print(df_data_age_income[mask])

       ConvertedCompYearly    Age
45                 60480.0  35-44
50                 25944.0  25-34
58                 22644.0  25-34
76                 45564.0  25-34
77                151263.0  35-44
...                    ...    ...
83425              19452.0  18-24
83428              41232.0  25-34
83431              11676.0  18-24
83432              80169.0  18-24
83436              90000.0  25-34

[11611 rows x 2 columns]


In [621]:
df_data_age_income = df_data_age_income[mask]

In [625]:
df_data_age_income1 = df_data_age_income.copy()

In [643]:
df_data_age_income1.to_csv('006_df_data_age_income1.csv', index=False)

In [627]:
grouped_df = df_data_age_income1.groupby("Age")

average_df = grouped_df.mean()

In [628]:
average_df

,ConvertedCompYearly
Age,
18-24,43758.228943
25-34,60962.367068
35-44,76911.641812
45-54,87229.578231
55-64,100102.974874
< 18,39841.117647
>= 65,95988.611111
No definido,77170.666667


In [644]:
df_flourish_006 = average_df.copy()

In [646]:
df_flourish_006.to_csv('006_df_flourish_006.csv', index=True)

### 2.7. ¿Cuáles son las tecnologías que permiten tener un mejor ingreso salarial anual?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [754]:
data_techs_best_income1 = data_test[['ConvertedCompYearly', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'MiscTechHaveWorkedWith',  'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith']]
data_techs_best_income1.head()

,ConvertedCompYearly,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith
45,60480.0,C#;C++;JavaScript;PowerShell;SQL;TypeScript,Microsoft SQL Server;PostgreSQL;Redis,Heroku;Microsoft Azure,ASP.NET Core ;React.js,.NET Core / .NET 5,Docker;Git;Kubernetes,Notepad++;Visual Studio;Visual Studio Code
50,25944.0,C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...,Couchbase;MariaDB;Microsoft SQL Server;MongoDB...,AWS;DigitalOcean;Microsoft Azure,Angular;ASP.NET;ASP.NET Core ;Express;Svelte,.NET Framework;.NET Core / .NET 5,Docker;Kubernetes,Notepad++;Visual Studio;Visual Studio Code
58,22644.0,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,Oracle,Heroku,Django;FastAPI;Vue.js,NumPy;Pandas;Torch/PyTorch,Docker;Git,IPython/Jupyter;Visual Studio Code
64,500000.0,HTML/CSS;JavaScript;Python,MySQL,AWS,Flask,Pandas,Git,Notepad++;PyCharm;Sublime Text
76,45564.0,Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...,Firebase;Microsoft SQL Server;MongoDB;MySQL;Po...,Google Cloud Platform;Microsoft Azure,Angular;Angular.js;ASP.NET;ASP.NET Core ;Djang...,.NET Framework;.NET Core / .NET 5;Apache Spark...,Docker;Git;Unity 3D,Android Studio;Eclipse;NetBeans;Notepad++;Visu...


In [755]:
data_techs_best_income1['AllTechs'] = data_techs_best_income1['LanguageHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['DatabaseHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['PlatformHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['WebframeHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['MiscTechHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['ToolsTechHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['NEWCollabToolsHaveWorkedWith'].map(str)
print (data_techs_best_income1)

       ConvertedCompYearly                             LanguageHaveWorkedWith  \
45                 60480.0        C#;C++;JavaScript;PowerShell;SQL;TypeScript   
50                 25944.0  C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...   
58                 22644.0          Bash/Shell;HTML/CSS;JavaScript;Python;SQL   
64                500000.0                         HTML/CSS;JavaScript;Python   
76                 45564.0  Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...   
...                    ...                                                ...   
83428              41232.0                      Bash/Shell;Node.js;TypeScript   
83431              11676.0  C#;Dart;HTML/CSS;Java;JavaScript;Kotlin;Node.j...   
83432              80169.0                                               Ruby   
83436              90000.0                                 Groovy;Java;Python   
83437             816816.0               Bash/Shell;JavaScript;Node.js;Python   

                           

C:\Users\GPBONI~1\AppData\Local\Temp/ipykernel_9952/782511894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_techs_best_income1['AllTechs'] = data_techs_best_income1['LanguageHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['DatabaseHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['PlatformHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['WebframeHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['MiscTechHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['ToolsTechHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['NEWCollabToolsHaveWorkedWith'].map(str)


In [757]:
df_data_techs_best_income = data_techs_best_income1[['ConvertedCompYearly', 'AllTechs']].copy()

In [759]:
df_data_techs_best_income1 = df_data_techs_best_income.copy()

In [760]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_techs_best_income1['ConvertedCompYearly'], 0.1)

print(df_data_techs_best_income1[mask])

       ConvertedCompYearly                                           AllTechs
45                 60480.0  C#;C++;JavaScript;PowerShell;SQL;TypeScript;Mi...
50                 25944.0  C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...
58                 22644.0  Bash/Shell;HTML/CSS;JavaScript;Python;SQL;Orac...
76                 45564.0  Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...
77                151263.0  HTML/CSS;Python;R;DynamoDB;AWS;Flask;Keras;Num...
...                    ...                                                ...
83425              19452.0  HTML/CSS;JavaScript;Node.js;TypeScript;DynamoD...
83428              41232.0  Bash/Shell;Node.js;TypeScript;Elasticsearch;Mo...
83431              11676.0  C#;Dart;HTML/CSS;Java;JavaScript;Kotlin;Node.j...
83432              80169.0  Ruby;MySQL;PostgreSQL;Google Cloud Platform;He...
83436              90000.0  Groovy;Java;Python;DynamoDB;Elasticsearch;Mong...

[11611 rows x 2 columns]


In [761]:
df_data_techs_best_income1 = df_data_techs_best_income1[mask]

In [762]:
df_data_techs_best_income1['ConvertedCompYearlyCategorical'] = 'ALTO'
df_data_techs_best_income1.loc[(df_data_techs_best_income1['ConvertedCompYearly'] >= 0) & (df_data_techs_best_income1['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_data_techs_best_income1.loc[(df_data_techs_best_income1['ConvertedCompYearly'] > 32747) & (df_data_techs_best_income1['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_data_techs_best_income1)

       ConvertedCompYearly                                           AllTechs  \
45                 60480.0  C#;C++;JavaScript;PowerShell;SQL;TypeScript;Mi...   
50                 25944.0  C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...   
58                 22644.0  Bash/Shell;HTML/CSS;JavaScript;Python;SQL;Orac...   
76                 45564.0  Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...   
77                151263.0  HTML/CSS;Python;R;DynamoDB;AWS;Flask;Keras;Num...   
...                    ...                                                ...   
83425              19452.0  HTML/CSS;JavaScript;Node.js;TypeScript;DynamoD...   
83428              41232.0  Bash/Shell;Node.js;TypeScript;Elasticsearch;Mo...   
83431              11676.0  C#;Dart;HTML/CSS;Java;JavaScript;Kotlin;Node.j...   
83432              80169.0  Ruby;MySQL;PostgreSQL;Google Cloud Platform;He...   
83436              90000.0  Groovy;Java;Python;DynamoDB;Elasticsearch;Mong...   

      ConvertedCompYearlyCa

In [763]:
df_data_techs_best_income1 = df_data_techs_best_income1[df_data_techs_best_income1['ConvertedCompYearlyCategorical'].isin(['ALTO', 'MEDIO'])]

In [765]:
df_data_techs_best_income1['AllTechs'] = df_data_techs_best_income1['AllTechs'].str.replace(' ', '')

In [766]:
df_data_techs_best_income1['AllTechs'] = df_data_techs_best_income1['AllTechs'].str.replace(';', ' ')

In [768]:
df_counts = df_data_techs_best_income1['AllTechs'].str.split(expand=True).stack().value_counts().rename_axis('Tech').reset_index(name='Count')

In [771]:
df_counts.head(10)

,Tech,Count
0,Git,8300
1,VisualStudioCode,7131
2,JavaScript,7057
3,Docker,5879
4,HTML/CSS,5821
5,SQL,5699
6,AWS,5066
7,C#,4717
8,TypeScript,4531
9,VisualStudio,4497


In [772]:
df_data_techs_best_income_007 = df_counts.head(10)

In [773]:
df_data_techs_best_income_007.to_csv('007_df_data_techs_best_income.csv', index=False)

### 2.8. ¿Cuántas tecnologías en promedio domina un programador profesional?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [792]:
data_techs_dev_pro1 = data_test[['DevType', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'MiscTechHaveWorkedWith',  'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith']]
data_techs_dev_pro1.head()

,DevType,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith
45,Desarrollador Escritorio,C#;C++;JavaScript;PowerShell;SQL;TypeScript,Microsoft SQL Server;PostgreSQL;Redis,Heroku;Microsoft Azure,ASP.NET Core ;React.js,.NET Core / .NET 5,Docker;Git;Kubernetes,Notepad++;Visual Studio;Visual Studio Code
50,Desarrollador full-stack,C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...,Couchbase;MariaDB;Microsoft SQL Server;MongoDB...,AWS;DigitalOcean;Microsoft Azure,Angular;ASP.NET;ASP.NET Core ;Express;Svelte,.NET Framework;.NET Core / .NET 5,Docker;Kubernetes,Notepad++;Visual Studio;Visual Studio Code
58,Desarrollador full-stack,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,Oracle,Heroku,Django;FastAPI;Vue.js,NumPy;Pandas;Torch/PyTorch,Docker;Git,IPython/Jupyter;Visual Studio Code
64,Desarrollador front-end,HTML/CSS;JavaScript;Python,MySQL,AWS,Flask,Pandas,Git,Notepad++;PyCharm;Sublime Text
76,Desarrollador front-end,Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...,Firebase;Microsoft SQL Server;MongoDB;MySQL;Po...,Google Cloud Platform;Microsoft Azure,Angular;Angular.js;ASP.NET;ASP.NET Core ;Djang...,.NET Framework;.NET Core / .NET 5;Apache Spark...,Docker;Git;Unity 3D,Android Studio;Eclipse;NetBeans;Notepad++;Visu...


In [793]:
data_techs_dev_pro1['AllTechs'] = data_techs_dev_pro1['LanguageHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['DatabaseHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['PlatformHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['WebframeHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['MiscTechHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['ToolsTechHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['NEWCollabToolsHaveWorkedWith'].map(str)
print (data_techs_dev_pro1)

                        DevType  \
45     Desarrollador Escritorio   
50     Desarrollador full-stack   
58     Desarrollador full-stack   
64      Desarrollador front-end   
76      Desarrollador front-end   
...                         ...   
83428          Ejecutivo Senior   
83431       Desarrollador móvil   
83432    Desarrollador back-end   
83436       Cientifico de datos   
83437    Desarrollador back-end   

                                  LanguageHaveWorkedWith  \
45           C#;C++;JavaScript;PowerShell;SQL;TypeScript   
50     C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...   
58             Bash/Shell;HTML/CSS;JavaScript;Python;SQL   
64                            HTML/CSS;JavaScript;Python   
76     Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...   
...                                                  ...   
83428                      Bash/Shell;Node.js;TypeScript   
83431  C#;Dart;HTML/CSS;Java;JavaScript;Kotlin;Node.j...   
83432                                  

C:\Users\GPBONI~1\AppData\Local\Temp/ipykernel_9952/1321581082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_techs_dev_pro1['AllTechs'] = data_techs_dev_pro1['LanguageHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['DatabaseHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['PlatformHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['WebframeHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['MiscTechHaveWorkedWith'].map(str) + ';' + data_techs_best_income1['ToolsTechHaveWorkedWith'].map(str) + ';' + data_techs_dev_pro1['NEWCollabToolsHaveWorkedWith'].map(str)


In [794]:
df_data_techs_dev_pro = data_techs_dev_pro1[['DevType', 'AllTechs']].copy()

In [796]:
df_data_techs_dev_pro = df_data_techs_dev_pro[df_data_techs_dev_pro['DevType'].isin(['Desarrollador full-stack', 'Desarrollador front-end', 'Desarrollador móvil', 'Desarrollador back-end', 'Desarrollador Escritorio', 'Desarrollador de QA o Test', 'Desarrollador de aplicaciones embebidas', 'Administrador de base de datos', 'Desarrollador de juegos o gráfico'])]

In [797]:
df_data_techs_dev_pro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12832 entries, 45 to 83437
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DevType   12832 non-null  object
 1   AllTechs  12832 non-null  object
dtypes: object(2)
memory usage: 300.8+ KB


In [858]:
df_data_techs_dev_pro1 = df_data_techs_dev_pro.copy()

In [860]:
df_data_techs_dev_pro1.to_csv('008_df_data_techs_dev_pro1.csv', index=True)

In [866]:
def convert_row_to_list(lst):
    return lst.split(';')

In [867]:
df_data_techs_dev_pro1['ListTechs'] = df_data_techs_dev_pro1['AllTechs'].apply(convert_row_to_list)

In [868]:
df_data_techs_dev_pro1['LenListTechs'] = df_data_techs_dev_pro1['ListTechs'].map(len)

In [871]:
df_flourish_008 = df_data_techs_dev_pro1[['DevType', 'LenListTechs']].copy()
df_flourish_008

,DevType,LenListTechs
45,Desarrollador Escritorio,20
50,Desarrollador full-stack,30
58,Desarrollador full-stack,17
64,Desarrollador front-end,11
76,Desarrollador front-end,50
...,...,...
83423,Desarrollador full-stack,26
83425,Desarrollador full-stack,14
83431,Desarrollador móvil,28
83432,Desarrollador back-end,21


In [879]:
grouped_df = df_flourish_008.groupby("DevType")

average_df_008 = round(grouped_df.mean())

In [882]:
df_flourish_008 = average_df_008.copy()

In [884]:
df_flourish_008.to_csv('008_df_flourish_008.csv', index=True)

### 2.9. ¿En qué rango de edad se inició la mayoría de los programadores en la programación?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [886]:
data_age1stcode_dev_pro1 = data_test[['Age1stCode']]
data_age1stcode_dev_pro1.head()

,Age1stCode
45,11-17
50,18-24
58,11-17
64,11-17
76,11-17


In [888]:
data_age1stcode_dev_pro1 = data_age1stcode_dev_pro1['Age1stCode'].value_counts().to_frame('counts').reset_index()

In [891]:
data_age1stcode_dev_pro1.to_csv('009_flourish_data.csv', index=False)

### 2.10. ¿Cuántos años como programadores se requiere para obtener un ingreso salarial alto?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [929]:
data_yearscode_high_income1 = data_test[['ConvertedCompYearly', 'YearsCode']]
data_yearscode_high_income1.head()

,ConvertedCompYearly,YearsCode
45,60480.0,22
50,25944.0,12
58,22644.0,5
64,500000.0,6
76,45564.0,12


In [930]:
df_data_yearscode_high_income = data_yearscode_high_income1.copy()

In [931]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_yearscode_high_income['ConvertedCompYearly'], 0.1)

print(df_data_yearscode_high_income[mask])

       ConvertedCompYearly YearsCode
45                 60480.0        22
50                 25944.0        12
58                 22644.0         5
76                 45564.0        12
77                151263.0        10
...                    ...       ...
83425              19452.0         5
83428              41232.0        12
83431              11676.0         9
83432              80169.0         5
83436              90000.0        10

[11611 rows x 2 columns]


In [932]:
df_data_yearscode_high_income = df_data_yearscode_high_income[mask]

In [933]:
df_data_yearscode_high_income['ConvertedCompYearlyCategorical'] = 'ALTO'

df_data_yearscode_high_income.loc[(df_data_yearscode_high_income['ConvertedCompYearly'] >= 0) & (df_data_yearscode_high_income['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_data_yearscode_high_income.loc[(df_data_yearscode_high_income['ConvertedCompYearly'] > 32747) & (df_data_yearscode_high_income['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_data_yearscode_high_income)

       ConvertedCompYearly YearsCode ConvertedCompYearlyCategorical
45                 60480.0        22                          MEDIO
50                 25944.0        12                           BAJO
58                 22644.0         5                           BAJO
76                 45564.0        12                          MEDIO
77                151263.0        10                           ALTO
...                    ...       ...                            ...
83425              19452.0         5                           BAJO
83428              41232.0        12                          MEDIO
83431              11676.0         9                           BAJO
83432              80169.0         5                          MEDIO
83436              90000.0        10                          MEDIO

[11611 rows x 3 columns]


In [971]:
df_data_yearscode_high_income.to_csv('010_df_flourish.csv', index=False)

In [953]:
df_data_yearscode_high_income['ConvertedCompYearlyCategorical'].value_counts()

MEDIO    5816
BAJO     2903
ALTO     2892
Name: ConvertedCompYearlyCategorical, dtype: int64

In [972]:
df_flourish_010 = df_data_yearscode_high_income[['YearsCode', 'ConvertedCompYearlyCategorical']].copy()

df_flourish_010.head()

,YearsCode,ConvertedCompYearlyCategorical
45,22,MEDIO
50,12,BAJO
58,5,BAJO
76,12,MEDIO
77,10,ALTO


In [974]:
df_flourish_010['YearsCode'] = pd.to_numeric(df_flourish_010['YearsCode'])

In [975]:
df_flourish_010.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11611 entries, 45 to 83436
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   YearsCode                       11611 non-null  int64 
 1   ConvertedCompYearlyCategorical  11611 non-null  object
dtypes: int64(1), object(1)
memory usage: 530.2+ KB


In [976]:
grouped_df_010 = df_flourish_010.groupby("ConvertedCompYearlyCategorical")

average_df_010 = round(grouped_df_010.mean())

In [977]:
average_df_010

,YearsCode
ConvertedCompYearlyCategorical,
ALTO,19.0
BAJO,10.0
MEDIO,14.0


In [978]:
average_df_010.to_csv('010_flourish_data.csv', index=True)

### 2.11. ¿Cuáles son los perfiles que registran los mejores ingresos?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [979]:
data_profiles_dev_high_income1 = data_test[['ConvertedCompYearly', 'DevType']].copy()
data_profiles_dev_high_income1.head()

,ConvertedCompYearly,DevType
45,60480.0,Desarrollador Escritorio
50,25944.0,Desarrollador full-stack
58,22644.0,Desarrollador full-stack
64,500000.0,Desarrollador front-end
76,45564.0,Desarrollador front-end


In [1010]:
df_data_profiles_dev_high_income = data_profiles_dev_high_income1.copy()

In [1011]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_data_profiles_dev_high_income['ConvertedCompYearly'], 0.1)

print(df_data_profiles_dev_high_income[mask])

       ConvertedCompYearly                   DevType
45                 60480.0  Desarrollador Escritorio
50                 25944.0  Desarrollador full-stack
58                 22644.0  Desarrollador full-stack
76                 45564.0   Desarrollador front-end
77                151263.0       Cientifico de datos
...                    ...                       ...
83425              19452.0  Desarrollador full-stack
83428              41232.0          Ejecutivo Senior
83431              11676.0       Desarrollador móvil
83432              80169.0    Desarrollador back-end
83436              90000.0       Cientifico de datos

[11611 rows x 2 columns]


In [1012]:
df_data_profiles_dev_high_income = df_data_profiles_dev_high_income[mask]

In [1013]:
df_data_profiles_dev_high_income['ConvertedCompYearlyCategorical'] = 'ALTO'

df_data_profiles_dev_high_income.loc[(df_data_profiles_dev_high_income['ConvertedCompYearly'] >= 0) & (df_data_profiles_dev_high_income['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_data_profiles_dev_high_income.loc[(df_data_profiles_dev_high_income['ConvertedCompYearly'] > 32747) & (df_data_profiles_dev_high_income['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_data_profiles_dev_high_income)

       ConvertedCompYearly                   DevType  \
45                 60480.0  Desarrollador Escritorio   
50                 25944.0  Desarrollador full-stack   
58                 22644.0  Desarrollador full-stack   
76                 45564.0   Desarrollador front-end   
77                151263.0       Cientifico de datos   
...                    ...                       ...   
83425              19452.0  Desarrollador full-stack   
83428              41232.0          Ejecutivo Senior   
83431              11676.0       Desarrollador móvil   
83432              80169.0    Desarrollador back-end   
83436              90000.0       Cientifico de datos   

      ConvertedCompYearlyCategorical  
45                             MEDIO  
50                              BAJO  
58                              BAJO  
76                             MEDIO  
77                              ALTO  
...                              ...  
83425                           BAJO  
83428          

In [1015]:
df_data_profiles_dev_high_income['ConvertedCompYearlyCategorical'].value_counts()

MEDIO    5816
BAJO     2903
ALTO     2892
Name: ConvertedCompYearlyCategorical, dtype: int64

In [1016]:
df_flourish_011 = df_data_profiles_dev_high_income[['DevType', 'ConvertedCompYearlyCategorical']].copy()

In [1018]:
df_flourish_011 = df_flourish_011[df_flourish_011['ConvertedCompYearlyCategorical'].isin(['ALTO'])]

In [1019]:
df_flourish_011.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2892 entries, 77 to 83372
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   DevType                         2892 non-null   object
 1   ConvertedCompYearlyCategorical  2892 non-null   object
dtypes: object(2)
memory usage: 67.8+ KB


In [1021]:
df_data_flourish_011 = df_flourish_011['DevType'].value_counts().to_frame('counts').reset_index()

In [1023]:
df_data_flourish_011 = df_data_flourish_011.head(10)

In [1024]:
df_data_flourish_011

,index,counts
0,Desarrollador full-stack,981
1,Desarrollador front-end,539
2,Desarrollador móvil,380
3,Desarrollador back-end,302
4,Desarrollador Escritorio,262
5,Ingeniero de datos,142
6,Cientifico de datos,70
7,Manager de Ingeniería,48
8,Otro,36
9,Especialista en DevOps,32


In [1025]:
df_data_flourish_011.to_csv('011_flourish_data.csv', index=False)

### 2.12. ¿Cuáles son las 10 tecnologías más usadas entre los programadores por países?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [390]:
data_10_techs_popular_dev_countries = data_test[['Country', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'MiscTechHaveWorkedWith',  'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith']]
data_10_techs_popular_dev_countries.head()

,Country,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith
45,Brazil,C#;C++;JavaScript;PowerShell;SQL;TypeScript,Microsoft SQL Server;PostgreSQL;Redis,Heroku;Microsoft Azure,ASP.NET Core ;React.js,.NET Core / .NET 5,Docker;Git;Kubernetes,Notepad++;Visual Studio;Visual Studio Code
50,Greece,C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...,Couchbase;MariaDB;Microsoft SQL Server;MongoDB...,AWS;DigitalOcean;Microsoft Azure,Angular;ASP.NET;ASP.NET Core ;Express;Svelte,.NET Framework;.NET Core / .NET 5,Docker;Kubernetes,Notepad++;Visual Studio;Visual Studio Code
58,Russian Federation,Bash/Shell;HTML/CSS;JavaScript;Python;SQL,Oracle,Heroku,Django;FastAPI;Vue.js,NumPy;Pandas;Torch/PyTorch,Docker;Git,IPython/Jupyter;Visual Studio Code
64,United States of America,HTML/CSS;JavaScript;Python,MySQL,AWS,Flask,Pandas,Git,Notepad++;PyCharm;Sublime Text
76,Poland,Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...,Firebase;Microsoft SQL Server;MongoDB;MySQL;Po...,Google Cloud Platform;Microsoft Azure,Angular;Angular.js;ASP.NET;ASP.NET Core ;Djang...,.NET Framework;.NET Core / .NET 5;Apache Spark...,Docker;Git;Unity 3D,Android Studio;Eclipse;NetBeans;Notepad++;Visu...


In [1029]:
data_10_techs_popular_dev_countries['AllTechs'] = data_10_techs_popular_dev_countries['LanguageHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['DatabaseHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['PlatformHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['WebframeHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['MiscTechHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['ToolsTechHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['NEWCollabToolsHaveWorkedWith'].map(str)
print (data_10_techs_popular_dev_countries)

                        Country  \
45                       Brazil   
50                       Greece   
58           Russian Federation   
64     United States of America   
76                       Poland   
...                         ...   
83428                    Brazil   
83431                  Pakistan   
83432                    Canada   
83436  United States of America   
83437                    Canada   

                                  LanguageHaveWorkedWith  \
45           C#;C++;JavaScript;PowerShell;SQL;TypeScript   
50     C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...   
58             Bash/Shell;HTML/CSS;JavaScript;Python;SQL   
64                            HTML/CSS;JavaScript;Python   
76     Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...   
...                                                  ...   
83428                      Bash/Shell;Node.js;TypeScript   
83431  C#;Dart;HTML/CSS;Java;JavaScript;Kotlin;Node.j...   
83432                                  

C:\Users\GPBONI~1\AppData\Local\Temp/ipykernel_9952/1489135702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_10_techs_popular_dev_countries['AllTechs'] = data_10_techs_popular_dev_countries['LanguageHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['DatabaseHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['PlatformHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['WebframeHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['MiscTechHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['ToolsTechHaveWorkedWith'].map(str) + ';' + data_10_techs_popular_dev_countries['NEWCollabToolsHaveWorkedWith'].map(str)


In [1030]:
df_data_10_techs_popular_dev_countries = data_10_techs_popular_dev_countries[['Country', 'AllTechs']].copy()

In [1031]:
df_data_10_techs_popular_dev_countries.head()

,Country,AllTechs
45,Brazil,C#;C++;JavaScript;PowerShell;SQL;TypeScript;Mi...
50,Greece,C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...
58,Russian Federation,Bash/Shell;HTML/CSS;JavaScript;Python;SQL;Orac...
64,United States of America,HTML/CSS;JavaScript;Python;MySQL;AWS;Flask;Pan...
76,Poland,Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...


In [1032]:
df_data_10_techs_popular_dev_countries['AllTechs'] = df_data_10_techs_popular_dev_countries['AllTechs'].str.replace(' ', '')

In [1033]:
df_data_10_techs_popular_dev_countries['AllTechs'] = df_data_10_techs_popular_dev_countries['AllTechs'].str.replace(';', ' ')

In [1034]:
df_counts = df_data_10_techs_popular_dev_countries['AllTechs'].str.split(expand=True).stack().value_counts().rename_axis('Tech').reset_index(name='Count')

In [1035]:
df_counts

,Tech,Count
0,Git,13828
1,VisualStudioCode,12030
2,JavaScript,11779
3,HTML/CSS,9714
4,Docker,9296
...,...,...
120,Erlang,128
121,Pulumi,121
122,COBOL,91
123,Crystal,87


In [391]:
data_10_techs_popular_dev_countries.to_csv('012_data_10_techs_popular_dev_countries.csv', index=False)

### 2.13. ¿Cuáles el sistema operativo más usado entre los encuestados?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1036]:
df_data_so_devs = data_test[['OpSys']].copy()

In [1038]:
df_data_so_devs.tail()

,OpSys
83428,MacOS
83431,Windows
83432,MacOS
83436,Windows
83437,MacOS


In [1039]:
df_data_so_devs['OpSys'].drop_duplicates().sort_values()

7037                        BSD
58                        Linux
77                        MacOS
464     Other (please specify):
45                      Windows
Name: OpSys, dtype: object

In [1042]:
df_data_so_devs['OpSys'] = df_data_so_devs['OpSys'].replace(['Other (please specify):'], 'Otro')

In [1043]:
df_data_so_devs['OpSys'].value_counts()

Windows    7293
MacOS      4255
Linux      2912
Otro         47
BSD          10
Name: OpSys, dtype: int64

In [1045]:
df_counts = df_data_so_devs['OpSys'].str.split(expand=True).stack().value_counts().rename_axis('OS').reset_index(name='Count')

In [1046]:
df_counts

,OS,Count
0,Windows,7293
1,MacOS,4255
2,Linux,2912
3,Otro,47
4,BSD,10


In [1047]:
df_counts.to_csv('013_flourish_data.csv', index=False)

### 2.14. ¿Qué proporción de programadores tiene algún desorden mental por país?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [394]:
data_devs_mental_health_countries = data_test[['Country', 'MentalHealth']]
data_devs_mental_health_countries.head()

,Country,MentalHealth
45,Brazil,Desorden emocional
50,Greece,Ninguna de las mencionadas
58,Russian Federation,Ninguna de las mencionadas
64,United States of America,Ninguna de las mencionadas
76,Poland,Ninguna de las mencionadas


In [1048]:
data_devs_mental_health_countries['MentalHealth'].value_counts()

Ninguna de las mencionadas             10924
Desorden de concentración o memoria     1303
Desorden emocional                       908
Desorden de ansiedad                     637
No definido                              538
Tipo de autismo                          207
Name: MentalHealth, dtype: int64

In [1100]:
df_data_devs_mental_health_countries = data_devs_mental_health_countries.copy()

In [1101]:
df_data_devs_mental_health_countries = df_data_devs_mental_health_countries[df_data_devs_mental_health_countries['MentalHealth'].isin(['Desorden de concentración o memoria', 'Desorden emocional', 'Desorden de ansiedad', 'Tipo de autismo'])]

In [1103]:
df_data_devs_mental_health_countries.head()

,Country,MentalHealth
45,Brazil,Desorden emocional
96,Germany,Desorden emocional
129,United States of America,Tipo de autismo
199,United States of America,Desorden de concentración o memoria
213,Russian Federation,Desorden de ansiedad


In [1091]:
df_data_flourish_014 = df_data_devs_mental_health_countries['Country'].value_counts().to_frame('counts').reset_index()

In [1095]:
df_data_flourish_014 = df_data_flourish_014.head(10)
df_data_flourish_014

,index,counts
0,United States of America,1027
1,United Kingdom of Great Britain and Northern I...,206
2,Brazil,194
3,Canada,140
4,India,134
5,Germany,112
6,Australia,85
7,Netherlands,84
8,Poland,57
9,Turkey,52


In [1111]:
df_data_flourish_014_best_ten = df_data_devs_mental_health_countries[df_data_devs_mental_health_countries['Country'].isin(['United States of America', 'United Kingdom of Great Britain and Northern Ireland', 'Brazil', 'Canada', 'India', 'Germany', 'Australia', 'Netherlands', 'Poland', 'Turkey'])]

In [1135]:
df = df_data_flourish_014_best_ten.copy()

In [1136]:
df

,Country,MentalHealth
45,Brazil,Desorden emocional
96,Germany,Desorden emocional
129,United States of America,Tipo de autismo
199,United States of America,Desorden de concentración o memoria
237,Germany,Desorden emocional
...,...,...
83319,Germany,Desorden de concentración o memoria
83342,United States of America,Desorden emocional
83347,Brazil,Desorden de ansiedad
83370,Brazil,Desorden de concentración o memoria


In [1138]:
df1 = pd.crosstab(df['Country'], df['MentalHealth'])
df1

MentalHealth,Desorden de concentración o memoria,Desorden emocional,Desorden de ansiedad,Tipo de autismo
Country,,,,
Australia,36,25,16,8
Brazil,67,52,64,11
Canada,71,30,30,9
Germany,38,43,21,10
India,42,51,38,3
Netherlands,42,15,11,16
Poland,14,26,7,10
Turkey,19,19,14,0
United Kingdom of Great Britain and Northern Ireland,64,77,42,23


In [1066]:
(df_data_devs_mental_health_countries.groupby(['Country', 'MentalHealth']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count') 
   .drop_duplicates(subset='Country'))

,Country,MentalHealth,count
0,United States of America,Desorden de concentración o memoria,573
3,United Kingdom of Great Britain and Northern I...,Desorden emocional,77
4,Canada,Desorden de concentración o memoria,71
5,Brazil,Desorden de concentración o memoria,67
9,India,Desorden emocional,51
...,...,...,...
295,Kuwait,Desorden de concentración o memoria,1
298,Luxembourg,Desorden de ansiedad,1
299,Malawi,Desorden de concentración o memoria,1
300,Maldives,Desorden emocional,1


In [1074]:
df_flourish_data_014 = (df_data_devs_mental_health_countries.groupby(['Country', 'MentalHealth']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count'))

In [1077]:
df_flourish_data_014 = df_flourish_data_014.sort_values('Country')

In [1094]:
df_data_flourish_014.head(10).to_csv('014_flourish_data_014.csv', index=False)

In [1140]:
df1.to_csv('014_flourish_data_014.csv', index=True)

### 2.15. ¿Cuáles son los países que tienen los mejores sueldos entre los programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1141]:
df_best_incomes_countries = data_test[['Country', 'ConvertedCompYearly']].copy()

In [1142]:
df_best_incomes_countries

,Country,ConvertedCompYearly
45,Brazil,60480.0
50,Greece,25944.0
58,Russian Federation,22644.0
64,United States of America,500000.0
76,Poland,45564.0
...,...,...
83428,Brazil,41232.0
83431,Pakistan,11676.0
83432,Canada,80169.0
83436,United States of America,90000.0


In [1143]:
def remove_outliers(df, q=0.05):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(df_best_incomes_countries['ConvertedCompYearly'], 0.1)

print(df_best_incomes_countries[mask])

                        Country  ConvertedCompYearly
45                       Brazil              60480.0
50                       Greece              25944.0
58           Russian Federation              22644.0
76                       Poland              45564.0
77                       Canada             151263.0
...                         ...                  ...
83425                   Finland              19452.0
83428                    Brazil              41232.0
83431                  Pakistan              11676.0
83432                    Canada              80169.0
83436  United States of America              90000.0

[11611 rows x 2 columns]


In [1145]:
df_best_incomes_countries_no_outliers = df_best_incomes_countries[mask]

In [1146]:
df_best_incomes_countries_no_outliers1 = df_best_incomes_countries_no_outliers.copy()

In [1148]:
df_best_incomes_countries_no_outliers1['ConvertedCompYearlyCategorical'] = 'ALTO'
df_best_incomes_countries_no_outliers1.loc[(df_best_incomes_countries_no_outliers1['ConvertedCompYearly'] >= 0) & (df_best_incomes_countries_no_outliers1['ConvertedCompYearly'] <= 32747), 'ConvertedCompYearlyCategorical'] = 'BAJO'
df_best_incomes_countries_no_outliers1.loc[(df_best_incomes_countries_no_outliers1['ConvertedCompYearly'] > 32747) & (df_best_incomes_countries_no_outliers1['ConvertedCompYearly'] <= 90000), 'ConvertedCompYearlyCategorical'] = 'MEDIO'

print(df_best_incomes_countries_no_outliers1)

                        Country  ConvertedCompYearly  \
45                       Brazil              60480.0   
50                       Greece              25944.0   
58           Russian Federation              22644.0   
76                       Poland              45564.0   
77                       Canada             151263.0   
...                         ...                  ...   
83425                   Finland              19452.0   
83428                    Brazil              41232.0   
83431                  Pakistan              11676.0   
83432                    Canada              80169.0   
83436  United States of America              90000.0   

      ConvertedCompYearlyCategorical  
45                             MEDIO  
50                              BAJO  
58                              BAJO  
76                             MEDIO  
77                              ALTO  
...                              ...  
83425                           BAJO  
83428          

In [1149]:
df_best_incomes_countries_no_outliers1['ConvertedCompYearlyCategorical'].value_counts()

MEDIO    5816
BAJO     2903
ALTO     2892
Name: ConvertedCompYearlyCategorical, dtype: int64

In [1151]:
df_best_incomes_countries_alto = df_best_incomes_countries_no_outliers1[df_best_incomes_countries_no_outliers1['ConvertedCompYearlyCategorical'] == 'ALTO']

In [1152]:
df_alto = df_best_incomes_countries_alto[['Country', 'ConvertedCompYearlyCategorical']].copy()

In [1154]:
df_flourish_015 = df_alto['Country'].value_counts().to_frame('counts').reset_index()

In [1156]:
df_flourish_015.head(10)

,index,counts
0,United States of America,1547
1,United Kingdom of Great Britain and Northern I...,244
2,Canada,166
3,Germany,107
4,Australia,106
5,Israel,82
6,Switzerland,81
7,Denmark,57
8,Netherlands,40
9,France,36


In [1157]:
df_flourish_015.head(10).to_csv('015_flourish_data.csv', index=False)

### 2.16. ¿Cuáles son los 10 lenguajes de programación más usados entre los programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1161]:
df_10_prog_languages_devs = data_test[['LanguageHaveWorkedWith']].copy()
df_10_prog_languages_devs.head()

,LanguageHaveWorkedWith
45,C#;C++;JavaScript;PowerShell;SQL;TypeScript
50,C#;HTML/CSS;JavaScript;Node.js;PowerShell;Type...
58,Bash/Shell;HTML/CSS;JavaScript;Python;SQL
64,HTML/CSS;JavaScript;Python
76,Bash/Shell;C#;Dart;Delphi;Go;HTML/CSS;Java;Jav...


In [1162]:
df_10_prog_languages_devs['LanguageHaveWorkedWith'] = df_10_prog_languages_devs['LanguageHaveWorkedWith'].str.replace(';', ' ')

In [1163]:
df_counts_016 = df_10_prog_languages_devs['LanguageHaveWorkedWith'].str.split(expand=True).stack().value_counts().rename_axis('Languages').reset_index(name='Count')

In [1164]:
df_counts_016.head(10)

,Languages,Count
0,JavaScript,11779
1,HTML/CSS,9714
2,SQL,9294
3,C#,7318
4,TypeScript,7261
5,Python,7225
6,Node.js,7066
7,Java,4855
8,Bash/Shell,4574
9,PHP,3524


In [1165]:
df_counts_016.head(10).to_csv('016_flourish_data.csv', index=False)

### 2.17. ¿Cuáles son las bases de datos más usadas entre los programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1171]:
df_10_databases = data_test[['DatabaseHaveWorkedWith']].copy()
df_10_databases.head()

,DatabaseHaveWorkedWith
45,Microsoft SQL Server;PostgreSQL;Redis
50,Couchbase;MariaDB;Microsoft SQL Server;MongoDB...
58,Oracle
64,MySQL
76,Firebase;Microsoft SQL Server;MongoDB;MySQL;Po...


In [1172]:
df_10_databases['DatabaseHaveWorkedWith'] = df_10_databases['DatabaseHaveWorkedWith'].str.replace(' ', '')

In [1173]:
df_10_databases['DatabaseHaveWorkedWith'] = df_10_databases['DatabaseHaveWorkedWith'].str.replace(';', ' ')

In [1174]:
df_counts_017 = df_10_databases['DatabaseHaveWorkedWith'].str.split(expand=True).stack().value_counts().rename_axis('Databases').reset_index(name='Count')

In [1175]:
df_counts_017.head(10)

,Databases,Count
0,PostgreSQL,7163
1,MySQL,7150
2,MicrosoftSQLServer,6553
3,SQLite,5442
4,MongoDB,5107
5,Redis,4507
6,Firebase,3032
7,Elasticsearch,2890
8,MariaDB,2704
9,Oracle,1921


In [1176]:
df_counts_017.head(10).to_csv('017_flourish_data.csv', index=False)

### 2.18. ¿Cuáles son las plataformas más usadas entre los programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1177]:
df_10_platforms = data_test[['PlatformHaveWorkedWith']].copy()
df_10_platforms.head()

,PlatformHaveWorkedWith
45,Heroku;Microsoft Azure
50,AWS;DigitalOcean;Microsoft Azure
58,Heroku
64,AWS
76,Google Cloud Platform;Microsoft Azure


In [1178]:
df_10_platforms['PlatformHaveWorkedWith'] = df_10_platforms['PlatformHaveWorkedWith'].str.replace(' ', '')

In [1179]:
df_10_platforms['PlatformHaveWorkedWith'] = df_10_platforms['PlatformHaveWorkedWith'].str.replace(';', ' ')

In [1181]:
df_counts_018 = df_10_platforms['PlatformHaveWorkedWith'].str.split(expand=True).stack().value_counts().rename_axis('Platform').reset_index(name='Count')

In [1182]:
df_counts_018.head(10)

,Platform,Count
0,AWS,8348
1,MicrosoftAzure,6738
2,GoogleCloudPlatform,4710
3,Heroku,3182
4,DigitalOcean,2829
5,IBMCloudorWatson,350
6,OracleCloudInfrastructure,212


In [1183]:
df_counts_018.to_csv('018_flourish_data.csv', index=False)

### 2.19. ¿Cuáles son los frameworks web más usados entre los programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1185]:
df_10_web_frameworks = data_test[['WebframeHaveWorkedWith']].copy()
df_10_web_frameworks.head()

,WebframeHaveWorkedWith
45,ASP.NET Core ;React.js
50,Angular;ASP.NET;ASP.NET Core ;Express;Svelte
58,Django;FastAPI;Vue.js
64,Flask
76,Angular;Angular.js;ASP.NET;ASP.NET Core ;Djang...


In [1186]:
df_10_web_frameworks['WebframeHaveWorkedWith'] = df_10_web_frameworks['WebframeHaveWorkedWith'].str.replace(' ', '')

In [1187]:
df_10_web_frameworks['WebframeHaveWorkedWith'] = df_10_web_frameworks['WebframeHaveWorkedWith'].str.replace(';', ' ')

In [1188]:
df_counts_019 = df_10_web_frameworks['WebframeHaveWorkedWith'].str.split(expand=True).stack().value_counts().rename_axis('Web framework').reset_index(name='Count')

In [1189]:
df_counts_019.head(10)

,Web framework,Count
0,React.js,6745
1,jQuery,5391
2,ASP.NETCore,5304
3,Angular,4506
4,ASP.NET,4169
5,Express,4048
6,Vue.js,3141
7,Flask,2873
8,Angular.js,2349
9,Django,2273


In [1190]:
df_counts_019.to_csv('019_flourish_data.csv', index=False)

### 2.20. ¿Cuáles son las herramientas tecnológicas más usadas entre los programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1192]:
df_10_data_misc_techs = data_test[['MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith']].copy()
df_10_data_misc_techs.head()

,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith
45,.NET Core / .NET 5,Docker;Git;Kubernetes
50,.NET Framework;.NET Core / .NET 5,Docker;Kubernetes
58,NumPy;Pandas;Torch/PyTorch,Docker;Git
64,Pandas,Git
76,.NET Framework;.NET Core / .NET 5;Apache Spark...,Docker;Git;Unity 3D


In [1193]:
df_10_data_misc_techs['AllMiscTechs'] = df_10_data_misc_techs['MiscTechHaveWorkedWith'].map(str) + ';' + df_10_data_misc_techs['ToolsTechHaveWorkedWith'].map(str)

In [1194]:
df_10_data_misc_techs.head()

,MiscTechHaveWorkedWith,ToolsTechHaveWorkedWith,AllMiscTechs
45,.NET Core / .NET 5,Docker;Git;Kubernetes,.NET Core / .NET 5;Docker;Git;Kubernetes
50,.NET Framework;.NET Core / .NET 5,Docker;Kubernetes,.NET Framework;.NET Core / .NET 5;Docker;Kuber...
58,NumPy;Pandas;Torch/PyTorch,Docker;Git,NumPy;Pandas;Torch/PyTorch;Docker;Git
64,Pandas,Git,Pandas;Git
76,.NET Framework;.NET Core / .NET 5;Apache Spark...,Docker;Git;Unity 3D,.NET Framework;.NET Core / .NET 5;Apache Spark...


In [1197]:
df_10_data_misc_techs['AllMiscTechs'] = df_10_data_misc_techs['AllMiscTechs'].str.replace(' ', '')

In [1198]:
df_10_data_misc_techs['AllMiscTechs'] = df_10_data_misc_techs['AllMiscTechs'].str.replace(';', ' ')

In [1200]:
df_counts_020 = df_10_data_misc_techs['AllMiscTechs'].str.split(expand=True).stack().value_counts().rename_axis('Tecnología').reset_index(name='# Programadores')

In [1201]:
df_counts_020.head(10)

,Tecnología,# Programadores
0,Git,13828
1,Docker,9296
2,.NETCore/.NET5,6046
3,.NETFramework,5697
4,NumPy,3807
5,Kubernetes,3709
6,Pandas,3634
7,Yarn,3617
8,ReactNative,2960
9,TensorFlow,2068


In [1202]:
df_counts_020.head(10).to_csv('020_flourish_data.csv', index=False)

### 2.21. ¿Cuáles son las herramientas colaborativas más usadas entre programadores?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1204]:
df_10_colab = data_test[['NEWCollabToolsHaveWorkedWith']].copy()
df_10_colab.head()

,NEWCollabToolsHaveWorkedWith
45,Notepad++;Visual Studio;Visual Studio Code
50,Notepad++;Visual Studio;Visual Studio Code
58,IPython/Jupyter;Visual Studio Code
64,Notepad++;PyCharm;Sublime Text
76,Android Studio;Eclipse;NetBeans;Notepad++;Visu...


In [1205]:
df_10_colab['NEWCollabToolsHaveWorkedWith'] = df_10_colab['NEWCollabToolsHaveWorkedWith'].str.replace(' ', '')

In [1206]:
df_10_colab['NEWCollabToolsHaveWorkedWith'] = df_10_colab['NEWCollabToolsHaveWorkedWith'].str.replace(';', ' ')

In [1207]:
df_counts_021 = df_10_colab['NEWCollabToolsHaveWorkedWith'].str.split(expand=True).stack().value_counts().rename_axis('Herramienta Colaborativa').reset_index(name='# Programadores')

In [1208]:
df_counts_021.head(10)

,Herramienta Colaborativa,# Programadores
0,VisualStudioCode,12030
1,VisualStudio,7183
2,Notepad++,4987
3,AndroidStudio,4291
4,IntelliJ,4242
5,Vim,3773
6,SublimeText,3080
7,PyCharm,3024
8,Xcode,2602
9,Eclipse,2176


In [1209]:
df_counts_021.head(10).to_csv('021_flourish_data.csv', index=False)

### 2.22. ¿Cuáles son los países con mayor número de programadores trabajando a tiempo completo?

Se seleccionarán los campos adecuados para responder a esta pregunta

In [1210]:
df_fulltime_employment = data_test[['Country', 'Employment']].copy()
df_fulltime_employment.head()

,Country,Employment
45,Brazil,Tiempo completo
50,Greece,Tiempo completo
58,Russian Federation,Tiempo completo
64,United States of America,Independiete
76,Poland,Tiempo completo


In [1213]:
df_fulltime_employment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14517 entries, 45 to 83437
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Country     14517 non-null  object
 1   Employment  14517 non-null  object
dtypes: object(2)
memory usage: 856.3+ KB


In [1211]:
df_fulltime_only = df_fulltime_employment[df_fulltime_employment['Employment'] == 'Tiempo completo']

In [1212]:
df_fulltime_only.head()

,Country,Employment
45,Brazil,Tiempo completo
50,Greece,Tiempo completo
58,Russian Federation,Tiempo completo
76,Poland,Tiempo completo
77,Canada,Tiempo completo


In [1215]:
df_flourish_022 = df_fulltime_only['Country'].value_counts().to_frame('# Programadores').reset_index()

In [1216]:
df_flourish_022.head(10)

,index,# Programadores
0,United States of America,2947
1,India,984
2,United Kingdom of Great Britain and Northern I...,859
3,Germany,611
4,Brazil,502
5,Canada,499
6,Australia,308
7,France,303
8,Spain,279
9,Netherlands,258


In [1218]:
df_flourish_022.head(10).to_csv('022_flourish_data.csv', index=False)